In [1]:
import os
import sys
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
import re
import json



warnings.filterwarnings(action='ignore')

## json 원본 데이터 셋 로딩

In [2]:
DATA_DIR = 'C:/workspace/nlp_study/data/Training/신문기사_train_original/'
FILE_NAME = 'train_original.json'

print('json 파일 로딩')
with open(DATA_DIR + FILE_NAME,'r', encoding='UTF8') as json_file:
    json_data = json.load(json_file)

# 읽어온 데이터 셋을 데이터프레임으로 변환

dataset = pd.DataFrame(json_data)

json 파일 로딩


In [3]:
# 데이터프레임의 column 중 documents에 저장되어있는 index와 text값만 활용해야함
# documents 컬럼에는 id, category, media_type, index, text 등 값들이 dict 타입으로 저장되어있음
# 각 key 값을 데이터 프레임의 컬럼으로, value 값은 행으로 만들어주기
print('데이터 프레임 변환')
column_list = dataset['documents'][0].keys()

for column in column_list:
    dataset[column] = dataset['documents'].apply(lambda x : x[column])

데이터 프레임 변환


In [4]:
dataset.head()

,name,delivery_date,documents,id,category,media_type,media_sub_type,media_name,size,char_count,publish_date,title,text,annotator_id,document_quality_scores,extractive,abstractive
0,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741778', 'category': '종합', 'media_t...",290741778,종합,online,지역지,광양신문,small,927,2018-01-05 18:54:55,"논 타작물 재배, 2월 말까지 신청하세요","[[{'index': 0, 'sentence': 'ha당 조사료 400만원…작물별 ...",11,"{'readable': 4, 'accurate': 3, 'informative': ...","[2, 3, 10]",[전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 ...
1,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741792', 'category': '종합', 'media_t...",290741792,종합,online,지역지,광양신문,small,764,2018-01-05 18:53:20,"여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성","[[{'index': 0, 'sentence': '8억 투입, 고소천사벽화·자산마을...",12,"{'readable': 4, 'accurate': 4, 'informative': ...","[2, 4, 11]",[여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천...
2,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741793', 'category': '스포츠', 'media_...",290741793,스포츠,online,지역지,광양신문,medium,1066,2018-01-05 18:52:15,“새해 정기 받고 올해는 반드시 일내자!”,"[[{'index': 0, 'sentence': '전남드래곤즈 해맞이 다짐…선수 영...",13,"{'readable': 4, 'accurate': 4, 'informative': ...","[3, 5, 7]",[전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구...
3,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741794', 'category': '정치', 'media_t...",290741794,정치,online,지역지,광양신문,small,746,2018-01-05 18:50:17,"농업인 역량 강화, 새해 실용교육 실시","[[{'index': 0, 'sentence': '11~24일, 매실·감·참다래 등...",12,"{'readable': 4, 'accurate': 4, 'informative': ...","[2, 3, 4]","[광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화..."
4,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741797', 'category': '종합', 'media_t...",290741797,종합,online,지역지,광양신문,small,978,2018-01-05 18:52:36,타이완 크루즈관광객 4천여명‘전남’온다,"[[{'index': 0, 'sentence': '홍콩 크루즈선사‘아쿠아리우스’ 4...",13,"{'readable': 4, 'accurate': 4, 'informative': ...","[3, 7, 4]",[올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입...


In [5]:
data = dataset.copy()
data.head()

,name,delivery_date,documents,id,category,media_type,media_sub_type,media_name,size,char_count,publish_date,title,text,annotator_id,document_quality_scores,extractive,abstractive
0,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741778', 'category': '종합', 'media_t...",290741778,종합,online,지역지,광양신문,small,927,2018-01-05 18:54:55,"논 타작물 재배, 2월 말까지 신청하세요","[[{'index': 0, 'sentence': 'ha당 조사료 400만원…작물별 ...",11,"{'readable': 4, 'accurate': 3, 'informative': ...","[2, 3, 10]",[전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 ...
1,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741792', 'category': '종합', 'media_t...",290741792,종합,online,지역지,광양신문,small,764,2018-01-05 18:53:20,"여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성","[[{'index': 0, 'sentence': '8억 투입, 고소천사벽화·자산마을...",12,"{'readable': 4, 'accurate': 4, 'informative': ...","[2, 4, 11]",[여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천...
2,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741793', 'category': '스포츠', 'media_...",290741793,스포츠,online,지역지,광양신문,medium,1066,2018-01-05 18:52:15,“새해 정기 받고 올해는 반드시 일내자!”,"[[{'index': 0, 'sentence': '전남드래곤즈 해맞이 다짐…선수 영...",13,"{'readable': 4, 'accurate': 4, 'informative': ...","[3, 5, 7]",[전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구...
3,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741794', 'category': '정치', 'media_t...",290741794,정치,online,지역지,광양신문,small,746,2018-01-05 18:50:17,"농업인 역량 강화, 새해 실용교육 실시","[[{'index': 0, 'sentence': '11~24일, 매실·감·참다래 등...",12,"{'readable': 4, 'accurate': 4, 'informative': ...","[2, 3, 4]","[광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화..."
4,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741797', 'category': '종합', 'media_t...",290741797,종합,online,지역지,광양신문,small,978,2018-01-05 18:52:36,타이완 크루즈관광객 4천여명‘전남’온다,"[[{'index': 0, 'sentence': '홍콩 크루즈선사‘아쿠아리우스’ 4...",13,"{'readable': 4, 'accurate': 4, 'informative': ...","[3, 7, 4]",[올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입...


In [6]:
data = dataset[['title','text','extractive','category']]
data

,title,text,extractive,category
0,"논 타작물 재배, 2월 말까지 신청하세요","[[{'index': 0, 'sentence': 'ha당 조사료 400만원…작물별 ...","[2, 3, 10]",종합
1,"여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성","[[{'index': 0, 'sentence': '8억 투입, 고소천사벽화·자산마을...","[2, 4, 11]",종합
2,“새해 정기 받고 올해는 반드시 일내자!”,"[[{'index': 0, 'sentence': '전남드래곤즈 해맞이 다짐…선수 영...","[3, 5, 7]",스포츠
3,"농업인 역량 강화, 새해 실용교육 실시","[[{'index': 0, 'sentence': '11~24일, 매실·감·참다래 등...","[2, 3, 4]",정치
4,타이완 크루즈관광객 4천여명‘전남’온다,"[[{'index': 0, 'sentence': '홍콩 크루즈선사‘아쿠아리우스’ 4...","[3, 7, 4]",종합
...,...,...,...,...
243978,"당진시, 찾아가는 청소년 노동인권 교육 진행","[[{'index': 0, 'sentence': '내달 12일까지, 노동인권 감수성...","[1, 2, 6]",종합
243979,"서산시, 2020년 성인문해교실 신청하세요!","[[{'index': 0, 'sentence': '마을별 10명 이상이면 신청가능,...","[5, 4, 8]",종합
243980,"서산시의회, 제247회 제2차 정례회 개회","[[{'index': 0, 'sentence': '2020년 예산안, 제4회 추경예...","[1, 3, 4]",종합
243981,"아산시, 온천 재활 헬스케어 힐링산업 본격 추진","[[{'index': 0, 'sentence': '전담기관 설립 추진, 타당성 검토...","[1, 2, 3]",종합


## text 컬럼 안에 있는 리스트에서 index, sentence 추출

In [7]:
sentence_list = []
extractive_target_list = []
category_list = []

for index in tqdm(range(len(data))):

    if None in data.loc[index,'extractive']:
        pass
    else:
        target_list = data['extractive'][index]
    
        for dict_list_index in range(len(data['text'][index])):
            for list_index in range(len(data['text'][index][dict_list_index])):
                text_data = data['text'][index][dict_list_index][list_index]['sentence']
                text_data = re.sub('\[{IMG[0-9][0-9]}]','',text_data)
                text_data = text_data.replace('\'','').replace('\"','').replace("’","").replace("‘","").replace("“","").replace("”","").replace('\\','')

                data_index = data['text'][index][dict_list_index][list_index]['index']

                if data_index in target_list:
                    extractive_target_list.append(1)
                else:
                    extractive_target_list.append(0)

                category_list.append(data['category'][index])
                sentence_list.append(text_data)

100%|██████████| 243983/243983 [00:46<00:00, 5237.64it/s]


In [10]:
print('데이터 저장')
dataset = pd.DataFrame()
dataset['sentence'] = sentence_list
dataset['target'] = extractive_target_list
dataset['category'] = category_list

데이터 저장


In [11]:
dataset = dataset.dropna()

In [12]:
dataset = dataset.reset_index(drop=True)
dataset

,sentence,target,category
0,ha당 조사료 400만원…작물별 차등 지원,0,종합
1,이성훈 sinawi@hanmail.net,0,종합
2,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...,1,종합
3,쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 ...,1,종합
4,올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698...,0,종합
...,...,...,...
3495550,용인비상에듀의 대입조기선발반 8주과정은 고교 3년간의 국영수 핵심 중요개념부터 마스...,0,종합
3495551,"1차 개강(12월 15일~12월 31일) 2주간은 국어,영어, 수학을 집중 학습하고...",1,종합
3495552,용인비상에듀 관계자는 무엇보다 학생들이 가장 어려워하고 변별력도 높은 과목인 수학시...,0,종합
3495553,또한 무엇보다 24시간 모든 생활이 학원에서 이루어지는 기숙학원의 특성상 스스로 관...,0,종합


In [13]:
dataset.to_csv(path_or_buf = DATA_DIR + 'dataset_final.csv',index=False)